In [1]:
import os
import glob
import tensorflow as tf
import numpy as np
import dask
import dask.array as da
from PIL import Image
from skimage.transform import resize
from matplotlib import pyplot as plt
import scipy
%matplotlib inline
np.set_printoptions(threshold=10000, linewidth=32)

In [2]:
y = np.asarray(sorted(glob.glob("/home/skyolia/JupyterProjects/data/BASE-CYTO/images etiquetées/*.bmp")))
x = np.asarray(sorted(glob.glob("/home/skyolia/JupyterProjects/data/BASE-CYTO/images originales/*.bmp")))

In [3]:
train = [ 0,  1,  2,  3,  4,  5,  6,  7,  8, 9,
 10, 11, 12, 13, 14, 15, 16, 17, 36, 37,
 38, 39, 40, 41, 42, 43, 44, 45, 46, 47,
 48, 49, 50, 51, 52, 53, 54, 55, 56, 57,
 58, 59, 60, 61, 62, 63, 64, 65, 66, 67,
 68, 69, 70, 71, 72, 73, 74, 75, 76, 77,
 78, 79, 80, 81, 82, 83, 84, 85, 86]
test = [18, 19, 20, 21, 22, 23, 24, 25, 26, 27,
 28, 29, 30, 31, 32, 33, 34, 35]
#train_x, train_y = x[train], y[train]
#test_x, test_y = x[test], y[test]
#train_x, train_y, test_x, test_y

In [3]:
rnd = np.random.choice(87, 61, False)
#train_x, train_y = x[train_idx], y[train_idx]

splitter = np.isin(np.arange(87), rnd, True)
train, test = np.where(splitter)[0], np.where(~splitter)[0]

np.sum(splitter), train, test

(61, array([ 0,  1,  3,  4,  6,  7,
        10, 12, 13, 14, 16, 17,
        18, 22, 25, 26, 27, 28,
        30, 32, 33, 34, 35, 36,
        38, 39, 40, 41, 42, 44,
        48, 50, 51, 52, 53, 54,
        55, 57, 58, 60, 61, 62,
        63, 64, 65, 66, 67, 68,
        70, 71, 72, 73, 74, 75,
        76, 78, 79, 80, 82, 83,
        86]), array([ 2,  5,  8,  9, 11, 15,
        19, 20, 21, 23, 24, 29,
        31, 37, 43, 45, 46, 47,
        49, 56, 59, 69, 77, 81,
        84, 85]))

In [4]:
def padding(obj, i):
    pad_size = np.asarray(obj) - np.asarray(i.shape)
    padded = np.pad(i, ((0, pad_size[0]), (0, pad_size[1]), (0, pad_size[2])), mode='edge')
    return padded

def extract_patch_single_image(i, crop_size):
    crops = []
    height, width = i.shape[0], i.shape[1]
    for h in range(0, height, 25):
        for w in range(0, width, 25):
            image = i[h:h+crop_size[0], w:w+crop_size[1], :]
            image = padding((crop_size[0], crop_size[1], image.shape[2]), image) if image.shape[0:2] != crop_size else image
            crops.append(image)
    return np.asarray(crops)

def load_save(path, mode, crop_size, new_shape=None):
    #image = Image.open(path).convert(mode)
    image = scipy.ndimage.imread(path, mode=mode)
    image = np.asarray(image)#/255.0
    image = image[:, :, np.newaxis] if image.ndim == 2 else image
    image = extract_patch_single_image(image, crop_size) if crop_size else image
    image = resize(image, new_shape, preserve_range = True) if new_shape else image
    return image#.astype(int)

def data_processing(data_paths, crop_size, new_shape, mode, chunk, stack=True):
    x = []
    for i in data_paths:
        image = dask.delayed(load_save)(i, mode, crop_size, new_shape=None)
        x.append(da.from_delayed(image, new_shape, dtype=int))
    res = da.stack(x, axis=0).rechunk(chunk) if stack else da.concatenate(x, axis=0).rechunk(chunk)
    return res

train_x = data_processing(x[train], (32, 32), (1271, 32, 32, 3), 'RGB', (15000, 32, 32, 3), False)
train_y = data_processing(y[train], (32, 32), (1271, 32, 32, 1), 'L', (15000, 32, 32, 1), False)
#colored_train_y = data_processing(masks[train], (32, 32), (7400, 32, 32, 3), 'RGB', (10000, 32, 32, 3), False)
test_x = data_processing(x[test], None, (768, 1024, 3), 'RGB', (1, 768, 1024, 3))
test_y = data_processing(y[test], None, (768, 1024, 1), 'L', (1, 768, 1024, 1))
colored_test_y = data_processing(y[test], None, (768, 1024, 3), 'RGB', (1, 768, 1024, 3))
train_x, train_y, test_x, test_y

(dask.array<rechunk-merge, shape=(87699, 32, 32, 3), dtype=int64, chunksize=(15000, 32, 32, 3)>,
 dask.array<rechunk-merge, shape=(87699, 32, 32, 1), dtype=int64, chunksize=(15000, 32, 32, 1)>,
 dask.array<stack, shape=(18, 768, 1024, 3), dtype=int64, chunksize=(1, 768, 1024, 3)>,
 dask.array<stack, shape=(18, 768, 1024, 1), dtype=int64, chunksize=(1, 768, 1024, 1)>)

In [9]:
#train_y = da.where(train_y==118, 117, train_y)
%time da.unique(train_y).compute(), da.unique(test_y).compute()

/home/skyolia/anaconda3/envs/tensorflow_gpuenv/lib/python3.6/site-packages/ipykernel_launcher.py:18: DeprecationWarning: `imread` is deprecated!
`imread` is deprecated in SciPy 1.0.0.
Use ``matplotlib.pyplot.imread`` instead.


CPU times: user 3.43 s, sys: 1.82 s, total: 5.25 s
Wall time: 6.08 s


(array([  0,  59, 117, 177],
       dtype=uint8), array([  0,  59, 117, 177],
       dtype=uint8))

In [5]:
def one_hot_labels_convertion(y):
    res = []
    for i in da.unique(y).compute():
        res.append(da.where(y==i, 1., 0))
    return da.concatenate(res, axis=3)

train_y = one_hot_labels_convertion(train_y)
test_y = one_hot_labels_convertion(test_y)
train_x, test_x = train_x/255., test_x/255.
train_x, train_y, test_x, test_y

/home/skyolia/anaconda3/envs/tensorflow_gpuenv/lib/python3.6/site-packages/ipykernel_launcher.py:18: DeprecationWarning: `imread` is deprecated!
`imread` is deprecated in SciPy 1.0.0.
Use ``matplotlib.pyplot.imread`` instead.


(dask.array<truediv, shape=(87699, 32, 32, 3), dtype=float64, chunksize=(15000, 32, 32, 3)>,
 dask.array<concatenate, shape=(87699, 32, 32, 4), dtype=float64, chunksize=(15000, 32, 32, 1)>,
 dask.array<truediv, shape=(18, 768, 1024, 3), dtype=float64, chunksize=(1, 768, 1024, 3)>,
 dask.array<concatenate, shape=(18, 768, 1024, 4), dtype=float64, chunksize=(1, 768, 1024, 1)>)

In [6]:
def build_block(input_layer, filters, norm=True, k=(3, 3)):
    layer = tf.keras.layers.Conv2D(filters, kernel_size=k, padding='same', use_bias=not norm, kernel_initializer='glorot_normal')(input_layer)
    if norm:
        layer = tf.keras.layers.BatchNormalization()(layer)
    layer = tf.keras.layers.Activation('elu')(layer)
    return layer

def build_unet(input_shape, n_filters, dropout=0.1):
    image_input = tf.keras.Input(shape=input_shape, name='input_layer')
    
    conv_1 = build_block(image_input, n_filters) #(32, 32, 16)
    conv_2 = build_block(conv_1, n_filters) #(32, 32, 16)
    pool_1 = tf.keras.layers.AveragePooling2D(padding='same')(conv_2)#(16, 16, 16)
    drop_1 = tf.keras.layers.SpatialDropout2D(dropout)(pool_1)
    
    conv_3 = build_block(drop_1, n_filters * 2)#(16, 16, 32)
    conv_4 = build_block(conv_3, n_filters * 2)#(16, 16, 32)
    pool_2 = tf.keras.layers.AveragePooling2D(padding='same')(conv_4)#(8, 8, 32)
    drop_2 = tf.keras.layers.SpatialDropout2D(dropout)(pool_2)
    
    conv_5 = build_block(drop_2, n_filters * 4)#(8, 8, 64)
    conv_6 = build_block(conv_5, n_filters * 4) #(8, 8, 64)
    pool_3 = tf.keras.layers.AveragePooling2D(padding='same')(conv_6)#(4, 4, 64)
    drop_3 = tf.keras.layers.SpatialDropout2D(dropout)(pool_3)
    
    conv_7 = build_block(drop_3, n_filters * 8)#(4, 4, 128)
    conv_8 = build_block(conv_7, n_filters * 8) #(4, 4, 128)
    
    upsp_1 = tf.keras.layers.UpSampling2D(size=(2, 2))(conv_8) #(-1, 8, 8, 128)
    upsp_1 = tf.keras.layers.concatenate([upsp_1, conv_6]) #(-1, 8, 8, 192)
    conv_9 = build_block(upsp_1, n_filters * 4) #(-1, 8, 8, 64)
    conv_10 = build_block(conv_9, n_filters * 4)
    drop_7 = tf.keras.layers.SpatialDropout2D(dropout)(conv_10)
    
    upsp_3 = tf.keras.layers.UpSampling2D(size=(2, 2))(drop_7) #(-1, 16, 16, 128)
    upsp_3 = tf.keras.layers.concatenate([upsp_3, conv_4]) #(-1, 16, 16, 192)
    conv_15 = build_block(upsp_3, n_filters * 2) #(-1, 16, 16, 64)
    conv_16 = build_block(conv_15, n_filters * 2)
    drop_7 = tf.keras.layers.SpatialDropout2D(dropout)(conv_16)
    
    upsp_4 = tf.keras.layers.UpSampling2D(size=(2, 2))(drop_7) #(-1, 32, 32, 64)
    upsp_4 = tf.keras.layers.concatenate([upsp_4, conv_2])#(-1, 32, 32, 92)
    conv_17 = build_block(upsp_4, n_filters)#(-1, 32, 32, 32)
    conv_18 = build_block(conv_17, n_filters)
    drop_8 = tf.keras.layers.SpatialDropout2D(dropout)(conv_18)
    
    output = tf.keras.layers.Conv2D(4, (1, 1), kernel_initializer='glorot_normal', activation='softmax')(drop_8)
    model = tf.keras.Model(inputs=image_input, outputs=output)
    return model
        
model = build_unet(input_shape=(None, None, 3), n_filters=16)
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_layer (InputLayer)        (None, None, None, 3 0                                            
__________________________________________________________________________________________________
conv2d (Conv2D)                 (None, None, None, 1 432         input_layer[0][0]                
__________________________________________________________________________________________________
batch_normalization (BatchNorma (None, None, None, 1 64          conv2d[0][0]                     
__________________________________________________________________________________________________
activation (Activation)         (None, None, None, 1 0           batch_normalization[0][0]        
__________________________________________________________________________________________________
conv2d_1 (

In [12]:
model.load_weights("day_1.weights.best.hdf5")
opt = tf.keras.optimizers.Adam() # 
model.compile(loss='categorical_crossentropy', optimizer=opt, metrics=['accuracy'])
scores = model.evaluate(test_x, test_y, batch_size=1)
scores

/home/skyolia/anaconda3/envs/tensorflow_gpuenv/lib/python3.6/site-packages/ipykernel_launcher.py:18: DeprecationWarning: `imread` is deprecated!
`imread` is deprecated in SciPy 1.0.0.
Use ``matplotlib.pyplot.imread`` instead.


26/26 [==============================] - 6s 242ms/step


[0.6664920054519405, 0.8167124516688861]

In [ ]:
epochs, batch_size, lr, filepath = 1000000, 256, 0.001, "day_1.weights.best.hdf5"
steps_per_epoch = int(np.ceil(train_y.shape[0]/batch_size))

data_gen_args = dict(horizontal_flip=True, vertical_flip=True)#, width_shift_range=0.1, height_shift_range=0.1
image_datagen = tf.keras.preprocessing.image.ImageDataGenerator(**data_gen_args)
mask_datagen = tf.keras.preprocessing.image.ImageDataGenerator(**data_gen_args)

# Provide the same seed and keyword arguments to the fit and flow methods
seed = 1
image_datagen.fit(train_x, augment=True, seed=seed)
mask_datagen.fit(train_y, augment=True, seed=seed)

image_generator = image_datagen.flow(x=train_x, batch_size=batch_size, seed=seed)
mask_generator = mask_datagen.flow(x=train_y, batch_size=batch_size, seed=seed)
train_generator = zip(image_generator, mask_generator)
test_generator = tf.keras.preprocessing.image.ImageDataGenerator().flow(x=test_x, y=test_y, batch_size=1)
checkpoint = tf.keras.callbacks.ModelCheckpoint(filepath, monitor='val_acc', verbose=1, save_best_only=True, mode='max')
tb = tf.keras.callbacks.TensorBoard(log_dir=os.getcwd())

opt = tf.keras.optimizers.Adam() # 
model.compile(loss='categorical_crossentropy', optimizer=opt, metrics=['accuracy'])
print(model.metrics_names)

model.fit_generator(train_generator,
                    steps_per_epoch=steps_per_epoch,
                    epochs=epochs,
                    verbose=1,
                    validation_data=test_generator,
                    validation_steps=int(np.ceil(test_y.shape[0]/batch_size)),
                    use_multiprocessing=False,
                    workers=12,
                    shuffle=True,
                    initial_epoch=0,
                    callbacks=[checkpoint, tb])
#tensorboard --logdir=/home/skyolia/JupyterProjects/segmentation/BASE-CYTO/

/home/skyolia/anaconda3/envs/tensorflow_gpuenv/lib/python3.6/site-packages/ipykernel_launcher.py:18: DeprecationWarning: `imread` is deprecated!
`imread` is deprecated in SciPy 1.0.0.
Use ``matplotlib.pyplot.imread`` instead.


['loss', 'acc']
Epoch 1/1000000
342/343 [============================>.] - ETA: 0s - loss: 60.8955 - acc: 0.9368
Epoch 00001: val_acc improved from -inf to 0.80564, saving model to day_1.weights.best.hdf5
343/343 [==============================] - 72s 209ms/step - loss: 60.7668 - acc: 0.9369 - val_loss: 0.8283 - val_acc: 0.8056
Epoch 2/1000000
342/343 [============================>.] - ETA: 0s - loss: 38.0923 - acc: 0.9522
Epoch 00002: val_acc improved from 0.80564 to 0.82429, saving model to day_1.weights.best.hdf5
343/343 [==============================] - 55s 160ms/step - loss: 38.0500 - acc: 0.9522 - val_loss: 0.7383 - val_acc: 0.8243
Epoch 3/1000000
342/343 [============================>.] - ETA: 0s - loss: 34.2368 - acc: 0.9552
Epoch 00003: val_acc improved from 0.82429 to 0.89253, saving model to day_1.weights.best.hdf5
343/343 [==============================] - 55s 161ms/step - loss: 34.1823 - acc: 0.9553 - val_loss: 0.3480 - val_acc: 0.8925
Epoch 4/1000000
342/343 [===========

342/343 [============================>.] - ETA: 0s - loss: 22.0563 - acc: 0.9688
Epoch 00030: val_acc did not improve from 0.90732
343/343 [==============================] - 55s 160ms/step - loss: 22.0337 - acc: 0.9687 - val_loss: 0.8718 - val_acc: 0.6777
Epoch 31/1000000
342/343 [============================>.] - ETA: 0s - loss: 22.1101 - acc: 0.9688
Epoch 00031: val_acc did not improve from 0.90732
343/343 [==============================] - 55s 160ms/step - loss: 22.0737 - acc: 0.9688 - val_loss: 0.3533 - val_acc: 0.8794
Epoch 32/1000000
342/343 [============================>.] - ETA: 0s - loss: 21.9674 - acc: 0.9689
Epoch 00032: val_acc did not improve from 0.90732
343/343 [==============================] - 55s 160ms/step - loss: 21.9408 - acc: 0.9689 - val_loss: 0.3224 - val_acc: 0.8977
Epoch 33/1000000
342/343 [============================>.] - ETA: 0s - loss: 21.7046 - acc: 0.9692
Epoch 00033: val_acc did not improve from 0.90732
343/343 [==============================] - 55s 160

Epoch 60/1000000
342/343 [============================>.] - ETA: 0s - loss: 19.2855 - acc: 0.9723
Epoch 00060: val_acc did not improve from 0.91565
343/343 [==============================] - 55s 160ms/step - loss: 19.2717 - acc: 0.9723 - val_loss: 0.4074 - val_acc: 0.8710
Epoch 61/1000000
342/343 [============================>.] - ETA: 0s - loss: 18.9366 - acc: 0.9728
Epoch 00061: val_acc did not improve from 0.91565
343/343 [==============================] - 55s 160ms/step - loss: 18.9031 - acc: 0.9728 - val_loss: 0.9505 - val_acc: 0.7069
Epoch 62/1000000
342/343 [============================>.] - ETA: 0s - loss: 18.9907 - acc: 0.9726
Epoch 00062: val_acc did not improve from 0.91565
343/343 [==============================] - 55s 161ms/step - loss: 18.9548 - acc: 0.9726 - val_loss: 0.3311 - val_acc: 0.8988
Epoch 63/1000000
342/343 [============================>.] - ETA: 0s - loss: 18.7984 - acc: 0.9729
Epoch 00063: val_acc did not improve from 0.91565
343/343 [========================

342/343 [============================>.] - ETA: 0s - loss: 16.9891 - acc: 0.9751
Epoch 00090: val_acc did not improve from 0.91565
343/343 [==============================] - 55s 160ms/step - loss: 16.9795 - acc: 0.9751 - val_loss: 0.5468 - val_acc: 0.8551
Epoch 91/1000000
342/343 [============================>.] - ETA: 0s - loss: 17.1943 - acc: 0.9748
Epoch 00091: val_acc did not improve from 0.91565
343/343 [==============================] - 55s 160ms/step - loss: 17.1638 - acc: 0.9749 - val_loss: 0.6690 - val_acc: 0.7863
Epoch 92/1000000
342/343 [============================>.] - ETA: 0s - loss: 17.0884 - acc: 0.9751
Epoch 00092: val_acc did not improve from 0.91565
343/343 [==============================] - 55s 160ms/step - loss: 17.0648 - acc: 0.9751 - val_loss: 0.8943 - val_acc: 0.7833
Epoch 93/1000000
342/343 [============================>.] - ETA: 0s - loss: 17.0818 - acc: 0.9749
Epoch 00093: val_acc did not improve from 0.91565
343/343 [==============================] - 55s 160

Epoch 120/1000000
342/343 [============================>.] - ETA: 0s - loss: 15.9625 - acc: 0.9764
Epoch 00120: val_acc did not improve from 0.91565
343/343 [==============================] - 55s 160ms/step - loss: 15.9371 - acc: 0.9764 - val_loss: 1.0659 - val_acc: 0.7279
Epoch 121/1000000
342/343 [============================>.] - ETA: 0s - loss: 16.1056 - acc: 0.9763
Epoch 00121: val_acc did not improve from 0.91565
343/343 [==============================] - 55s 161ms/step - loss: 16.0832 - acc: 0.9763 - val_loss: 0.3843 - val_acc: 0.8758
Epoch 122/1000000
342/343 [============================>.] - ETA: 0s - loss: 15.8333 - acc: 0.9766
Epoch 00122: val_acc did not improve from 0.91565
343/343 [==============================] - 55s 160ms/step - loss: 15.8204 - acc: 0.9765 - val_loss: 0.3731 - val_acc: 0.8740
Epoch 123/1000000
342/343 [============================>.] - ETA: 0s - loss: 15.7220 - acc: 0.9767
Epoch 00123: val_acc did not improve from 0.91565
343/343 [====================

Epoch 150/1000000
342/343 [============================>.] - ETA: 0s - loss: 14.8446 - acc: 0.9779
Epoch 00150: val_acc did not improve from 0.91565
343/343 [==============================] - 55s 160ms/step - loss: 14.8215 - acc: 0.9779 - val_loss: 0.3924 - val_acc: 0.8807
Epoch 151/1000000
342/343 [============================>.] - ETA: 0s - loss: 14.8595 - acc: 0.9779
Epoch 00151: val_acc did not improve from 0.91565
343/343 [==============================] - 55s 160ms/step - loss: 14.8351 - acc: 0.9779 - val_loss: 0.4210 - val_acc: 0.8806
Epoch 152/1000000
342/343 [============================>.] - ETA: 0s - loss: 14.7869 - acc: 0.9779
Epoch 00152: val_acc did not improve from 0.91565
343/343 [==============================] - 55s 160ms/step - loss: 14.7743 - acc: 0.9779 - val_loss: 0.5486 - val_acc: 0.8444
Epoch 153/1000000
342/343 [============================>.] - ETA: 0s - loss: 14.9116 - acc: 0.9777
Epoch 00153: val_acc did not improve from 0.91565
343/343 [====================

Epoch 180/1000000
342/343 [============================>.] - ETA: 0s - loss: 14.2174 - acc: 0.9786
Epoch 00180: val_acc did not improve from 0.91676
343/343 [==============================] - 55s 159ms/step - loss: 14.1986 - acc: 0.9786 - val_loss: 0.4154 - val_acc: 0.8794
Epoch 181/1000000
342/343 [============================>.] - ETA: 0s - loss: 13.9479 - acc: 0.9790
Epoch 00181: val_acc did not improve from 0.91676
343/343 [==============================] - 55s 160ms/step - loss: 13.9253 - acc: 0.9790 - val_loss: 0.4047 - val_acc: 0.8881
Epoch 182/1000000
342/343 [============================>.] - ETA: 0s - loss: 13.9500 - acc: 0.9790
Epoch 00182: val_acc did not improve from 0.91676
343/343 [==============================] - 55s 160ms/step - loss: 13.9335 - acc: 0.9790 - val_loss: 0.4370 - val_acc: 0.8834
Epoch 183/1000000
342/343 [============================>.] - ETA: 0s - loss: 13.9350 - acc: 0.9791
Epoch 00183: val_acc did not improve from 0.91676
343/343 [====================

Epoch 210/1000000
342/343 [============================>.] - ETA: 0s - loss: 13.2740 - acc: 0.9799
Epoch 00210: val_acc did not improve from 0.91676
343/343 [==============================] - 55s 159ms/step - loss: 13.2553 - acc: 0.9799 - val_loss: 0.3413 - val_acc: 0.9111
Epoch 211/1000000
342/343 [============================>.] - ETA: 0s - loss: 13.2363 - acc: 0.9800
Epoch 00211: val_acc did not improve from 0.91676
343/343 [==============================] - 55s 160ms/step - loss: 13.2160 - acc: 0.9800 - val_loss: 0.3621 - val_acc: 0.9048
Epoch 212/1000000
342/343 [============================>.] - ETA: 0s - loss: 13.1637 - acc: 0.9800
Epoch 00212: val_acc did not improve from 0.91676
343/343 [==============================] - 55s 160ms/step - loss: 13.1532 - acc: 0.9800 - val_loss: 0.4340 - val_acc: 0.8907
Epoch 213/1000000
342/343 [============================>.] - ETA: 0s - loss: 13.2803 - acc: 0.9799
Epoch 00213: val_acc did not improve from 0.91676
343/343 [====================

Epoch 240/1000000
342/343 [============================>.] - ETA: 0s - loss: 12.7557 - acc: 0.9806
Epoch 00240: val_acc did not improve from 0.91676
343/343 [==============================] - 55s 160ms/step - loss: 12.7460 - acc: 0.9806 - val_loss: 0.4181 - val_acc: 0.8894
Epoch 241/1000000
342/343 [============================>.] - ETA: 0s - loss: 12.7847 - acc: 0.9806
Epoch 00241: val_acc did not improve from 0.91676
343/343 [==============================] - 55s 160ms/step - loss: 12.7709 - acc: 0.9806 - val_loss: 0.5477 - val_acc: 0.8576
Epoch 242/1000000
342/343 [============================>.] - ETA: 0s - loss: 12.8486 - acc: 0.9805
Epoch 00242: val_acc did not improve from 0.91676
343/343 [==============================] - 55s 160ms/step - loss: 12.8353 - acc: 0.9805 - val_loss: 0.3673 - val_acc: 0.9098
Epoch 243/1000000
342/343 [============================>.] - ETA: 0s - loss: 12.8329 - acc: 0.9805
Epoch 00243: val_acc did not improve from 0.91676
343/343 [====================

In [6]:
#model.save('model.h5')
#del model
loaded_model = tf.keras.models.load_model('model.h5')
scores = loaded_model.evaluate(test_x, test_y, batch_size=1)
scores

/home/skyolia/anaconda3/envs/tensorflow_gpuenv/lib/python3.6/site-packages/ipykernel_launcher.py:18: DeprecationWarning: `imread` is deprecated!
`imread` is deprecated in SciPy 1.0.0.
Use ``matplotlib.pyplot.imread`` instead.


18/18 [==============================] - 7s 370ms/step


[0.5619055804693036, 0.878582066959805]